In [ ]:
import pandas as pd
import numpy as np
from sklearn import metrics
from functools import partial

from commons import *
from classifier import *

In [ ]:
train = pd.DataFrame.from_csv(trainFile,sep=";",index_col=None)
test = pd.DataFrame.from_csv(testFile,sep=";",index_col=None)

In [ ]:
x_train = train

In [ ]:
y_train = train[col_decision]

In [ ]:
#base_proba = {
#    buy:0.8533510965759615,
#    hold:0.3147490211779629,
#    sell:0.4252465377050152
#}

confidence_map = {
    CONF_NO_EXPERT_MODEL:0.25,
    CONF_NO_EXPERT_SYMBOL_MODEL:0.5,
    CONF_ALL:1
}

In [ ]:
clf = Classifier(base_proba,confidence_map)

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold
import numpy as np
def Kcrossvalidation_transformed(clf, X, Y, X_transformed, K = 10):
    score = np.zeros(K)
    kf = StratifiedKFold(n_splits=K,shuffle=True)
    i = 0
    for train, test in kf.split(X,Y):
        X_train, X_test, y_train, y_test = X.iloc[train], X.iloc[test], Y[train], Y[test]
        
        X_train_t = X_transformed[X_transformed[col_trans].isin(train)]
        X_test_t = X_transformed[X_transformed[col_trans].isin(test)]
        
        clf.fit_transformed(X_train_t)
        
        y_test_pred = clf.predict_transformed(X_test_t)
        score[i] = accuracy(y_test,y_test_pred)
        i += 1
    return np.mean(score)

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold
import numpy as np
def Kcrossvalidation(clf, X, Y, K = 10):
    score = np.zeros(K)
    kf = StratifiedKFold(n_splits=K,shuffle=True)
    i = 0
    for train, test in kf.split(X,Y):
        X_train, X_test, y_train, y_test = X.iloc[train], X.iloc[test], Y[train], Y[test]
        clf.fit(X_train, y_train)
        
        y_test_pred = clf.predict(X_test)
        score[i] = accuracy(y_test,y_test_pred)
        i += 1
    return np.mean(score)

In [ ]:
subset = range(0,2500)
x_train_s = x_train.iloc[subset]
y_train_s = y_train[subset]

ex = Extractor()

x_train_t_s = ex.transform(x_train_s)
x_train_t_s[col_decision] = y_train_s[x_train_t_s[col_trans]].map(vmap).tolist()

In [ ]:
Kcrossvalidation_transformed(clf,x_train_s,y_train_s,x_train_t_s,K=2)

In [ ]:
x_train.head(10)

In [ ]:
len(x_train)

In [ ]:
clf.fit(x_train, y_train)

In [ ]:
y_pred_train=clf.predict(test)

In [ ]:
y_pred_train

In [ ]:
pd.DataFrame(y_pred_train).to_csv('solution.csv',header=False,index=False)

In [ ]:
import array, random
from deap import creator, base, tools, algorithms

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register("attr_float", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual,  toolbox.attr_float, n=int(6))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

In [ ]:
def parse_base_proba(individual):
    return {
        buy:individual[3],
        hold:individual[4],
        sell:individual[5]
    }

def parse_confidence_map(individual):
    return {
        CONF_NO_EXPERT_MODEL:individual[0],
        CONF_NO_EXPERT_SYMBOL_MODEL:individual[1],
        CONF_ALL:individual[2]
    }

def evalOneMax(individual):
    clf = Classifier(parse_base_proba(individual),parse_confidence_map(individual))
    return (Kcrossvalidation_transformed(clf,x_train_s,y_train_s,x_train_t_s,K=2),)

toolbox.register("evaluate", evalOneMax)